In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import date

###TO DO MAKE THE CODE MORE MODULAR AND REMOVE THE SHITTY MAGIC VARIABLE
### I WILL NOT REFACTOR THIS UNTIL THE MAIN FEATURE IS DONE

def convert_snapshot(data, desired_key = None):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    if (desired_key is not None):
        data_dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(data_dictionary))

    return list(data_dictionary)


def fill_non_existent_column(df, is_caregiver=False):
    """
    This fill the df with unique problems column i
    """
    if is_caregiver is False:
        new_column = ['ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan',
                      'Gangguan-stres-pascatrauma', 'Skizofrenia']
    else:
        new_column = ['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan',
                      'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']

    for col in new_column:
        if (col in df.columns):
            pass
        else:
            df[col] = 0


def df_to_dataset(dataframe, shuffle=True, batch_size=32, text_processed=False):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    if(text_processed):
        processed_text = dataframe.pop('text_processed')
        ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
        ds_processed = tf.data.Dataset.from_tensors(dict(processed_text))
        ds = tf.data.Dataset.zip((ds,ds_processed))
    else:
        ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds



def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

def convert_categorical_data(df, col, is_caregiver = False):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(not is_caregiver):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)


def change_column_name(df, old_name, new_name):
    """
    replace list of column name in old_name to new colum name in new_name
    the order of old_name and new_name need to be exact
    """
    res = {old_name[i]: new_name[i] for i in range(len(new_name))}
    df.rename(columns=res, inplace=True)


def Merge(dict1, dict2):
    """
    Python code to merge dict
    """
    res = {**dict1, **dict2}
    return res


def unpack_caregiver_snapshot(data):
    """

    :param data: data is a snapshot of chatroom collection
    :return: list of caregiver in form of dictionary
    """
    desired_keys = ["problems", "birthday", "gender", "caregiver_id", "text"]
    # this unpack the value of snapshot list and turn the snapshot into dictionary also add aditional key for caregiver_id
    caregiver_dict_unpacked = map(lambda feature: Merge(feature.to_dict(), {'caregiver_id': feature.id}), data)

    # this unpack the snapshot in 'caregiver' snapshot and add them back to main dictionary
    caregiver_dict = map(lambda feature: Merge(feature, feature.pop('caregiver').get().to_dict()),
                         caregiver_dict_unpacked)

    #check for room capacity
    caregiver_dict = filter(lambda feature: feature['max_user'] > len(feature['users']) , caregiver_dict)

    # this seperate desired keys from unused keys
    caregiver_dict = map(lambda x: dict((k, x[k]) for k in desired_keys if k in x), caregiver_dict)

    # this convert birthday stamp into interger age
    caregiver_dict = map(lambda feature: Merge(feature, {'age': age(feature.pop('birthday'))}), caregiver_dict)

    return list(caregiver_dict)

def convert_caregiver_dictList_to_df(dictionary):
    """
    :param dictionary: list of dictionary
    :return: dataframe of the said dictionary
    """
    # Creates DataFrame.
    df = pd.DataFrame(dictionary)
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Caregiver_Tipe_Masalah','Caregiver_Gender','CAREGIVER_ID','Caregiver_Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Caregiver_Tipe_Masalah", is_caregiver = True)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = True)
    return df



In [2]:

from firebase_admin import credentials, firestore, initialize_app
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()


In [3]:

MAIN_TEXT_MODEL = tf.keras.models.load_model('saved_manual_model/text_query_v1')
MAIN_USER_MODEL = tf.keras.models.load_model('saved_manual_model/user_query_v1')
MAIN_CAREGIVER_MODEL = tf.keras.models.load_model('saved_manual_model/caregiver_query_v1')

c:\users\asus\anaconda3\envs\notebook\lib\site-packages\keras\saving\saved_model\load.py:115: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  metadata.ParseFromString(file_content)


In [4]:

db_ref = db.collection('chat_room_pref')
data = db_ref.get()

In [5]:
import tensorflow_recommenders as tfrs
data = unpack_caregiver_snapshot(data)
data = convert_caregiver_dictList_to_df(data)
caregiver_ds = df_to_dataset(data)
TEXT_INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_TEXT_MODEL)

In [6]:
TEXT_INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_TEXT_MODEL)



INDEX = tfrs.layers.factorized_top_k.BruteForce(MAIN_USER_MODEL)


In [7]:
# update this
TEXT_INDEX.index_from_dataset(
    caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_TEXT_MODEL([features['text']])))
)

# update_this
INDEX.index_from_dataset(
    caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))
)

ValueError: in user code:

    File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_3952\1999664232.py", line 8, in None  *
        lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features))

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (3 total):
        * {'Caregiver_Gender': <tf.Tensor 'inputs_12:0' shape=(None,) dtype=string>, 'CAREGIVER_ID': <tf.Tensor 'inputs:0' shape=(None,) dtype=string>, 'text': <tf.Tensor 'inputs_13:0' shape=(None,) dtype=string>, 'Caregiver_Age': <tf.Tensor 'inputs_11:0' shape=(None,) dtype=int64>, 'Caregiver-Anxiety': <tf.Tensor 'inputs_2:0' shape=(None,) dtype=int32>, 'Caregiver-Depresi': <tf.Tensor 'inputs_3:0' shape=(None,) dtype=int32>, 'Caregiver-Gangguan-kecemasan': <tf.Tensor 'inputs_4:0' shape=(None,) dtype=int32>, 'Caregiver-Gangguan-stress-pascatrauma': <tf.Tensor 'inputs_7:0' shape=(None,) dtype=int32>, 'Caregiver-Kekerasan': <tf.Tensor 'inputs_8:0' shap

In [ ]:
a = ['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan',
         'Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el, ([value[ix]])) for ix, el in enumerate(a))
print(dictionary_Test)

a, titles = INDEX(dictionary_Test, k=1)
print(f"Top recommendations: {titles[0]}")

a, titles = TEXT_INDEX(np.array(["AAAA sa sayang"]), k=1)

print(f"Top recommendations: {titles[0]}")

print(MAIN_TEXT_MODEL(np.array(["aa sss"])))

In [8]:
def unpack_user_snapshot(data):
    print(type(data))
    if (type(data) != list):
        data = [data]
    print(type(data))
    data = convert_snapshot(data, desired_key=["problems", "gender", "birthday", "text"])
    data = map(lambda feature: Merge(feature, {'age': age(feature.pop('birthday'))}), data)
    
    return data

def convert_user_dictList_to_df(dictionary):
    """
    :param dictionary: list of dictionary
    :return: dataframe of the said dictionary
    """
    # Creates DataFrame.
    df = pd.DataFrame(dictionary)
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Tipe_Masalah','Gender','CAREGIVER_ID','Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Tipe_Masalah", is_caregiver = False)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = False)
    return df



In [9]:
db_user = db.collection('users').document("4R3TTPINWhPzUvwIUEXbldtNAY72")


In [10]:
data_ref = db_user.get()

In [11]:
data = data_ref
print(data.to_dict())

{'birthday': DatetimeWithNanoseconds(2000, 11, 16, 16, 0, tzinfo=datetime.timezone.utc), 'text': 'aku stress', 'img': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/John_Cena_July_2018.jpg/640px-John_Cena_July_2018.jpg', 'rating': [4, 5], 'name': 'Dhiva', 'problems': ' Anxiety Kekerasan Masalah-Tidur', 'point': 0, 'gender': 'Pria'}


In [12]:
data = list(unpack_user_snapshot(data))
print(data)

<class 'google.cloud.firestore_v1.base_document.DocumentSnapshot'>
<class 'list'>
[{'problems': ' Anxiety Kekerasan Masalah-Tidur', 'gender': 'Pria', 'text': 'aku stress', 'age': 21}]


In [13]:
data = convert_user_dictList_to_df(data)
data.head(4)

,Gender,text,Age,Anxiety,Kekerasan,Masalah-Tidur,ADHD-Hiperaktif-dan-kurang-fokus,Depresi,Gangguan-kecemasan,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia
0,Pria,aku stress,21,1,1,1,0,0,0,0,0,0


In [14]:
data =data.to_dict('records')[0]

In [15]:
for k, v in data.items():
    data[k] = [v]

In [16]:
a, titles = INDEX(data, k=1)
print(f"Top recommendations: {titles[0]}")

AttributeError: Exception encountered when calling layer "brute_force_2" (type BruteForce).

'BruteForce' object has no attribute '_candidates'

Call arguments received:
  • queries={'Gender': ["'Pria'"], 'text': ["'aku stress'"], 'Age': ['tf.Tensor(shape=(), dtype=int32)'], 'Anxiety': ['tf.Tensor(shape=(), dtype=int32)'], 'Kekerasan': ['tf.Tensor(shape=(), dtype=int32)'], 'Masalah-Tidur': ['tf.Tensor(shape=(), dtype=int32)'], 'ADHD-Hiperaktif-dan-kurang-fokus': ['tf.Tensor(shape=(), dtype=int32)'], 'Depresi': ['tf.Tensor(shape=(), dtype=int32)'], 'Gangguan-kecemasan': ['tf.Tensor(shape=(), dtype=int32)'], 'Gangguan-makan': ['tf.Tensor(shape=(), dtype=int32)'], 'Gangguan-stres-pascatrauma': ['tf.Tensor(shape=(), dtype=int32)'], 'Skizofrenia': ['tf.Tensor(shape=(), dtype=int32)']}
  • k=1

In [17]:
from flask import json

In [18]:
test = {
    'recommendation': titles[0].numpy().tolist()
}

NameError: name 'titles' is not defined

In [19]:
type(test['recommendation'])

NameError: name 'test' is not defined

In [20]:
test = list(map(lambda string: string.decode("utf-8"), test['recommendation']))

NameError: name 'test' is not defined

In [21]:
test

NameError: name 'test' is not defined

In [22]:
db_chat_room_pref = db.collection('chat_room_pref')

In [24]:
chat_data = db_chat_room_pref.get()

In [146]:

data = unpack_caregiver_snapshot(chat_data)
data = convert_caregiver_dictList_to_df(data)

In [147]:
data

,Caregiver_Gender,CAREGIVER_ID,text,Caregiver_Age,Caregiver-Anxiety,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-stress-pascatrauma,Caregiver-Kekerasan,Caregiver-Masalah-Tidur,Caregiver-Skizofrenia,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma
0,Pria,HUDwzgELY1OEzwieoHXK,aku stress,21,1,0,0,0,1,1,0,0,0,0
1,Perempuan,49cugkYNFakCCkId9Iai,Lagi menunggu ujian sbmptn,21,0,0,1,0,0,0,0,0,0,0
2,Pria,db5wFkLGR4pb7ongAd3e,Kekerasan fisik,22,0,1,0,1,0,0,0,0,0,0
3,Perempuan,gSshnDOP0BbKXuaYTHZI,Kesulitan dalam berkonsentrasi dan mengingat s...,21,0,0,0,0,0,0,1,0,0,0


In [148]:
MAIN_DATAFRAME = data

In [149]:
MAIN_DATAFRAME

,Caregiver_Gender,CAREGIVER_ID,text,Caregiver_Age,Caregiver-Anxiety,Caregiver-Depresi,Caregiver-Gangguan-kecemasan,Caregiver-Gangguan-stress-pascatrauma,Caregiver-Kekerasan,Caregiver-Masalah-Tidur,Caregiver-Skizofrenia,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma
0,Pria,HUDwzgELY1OEzwieoHXK,aku stress,21,1,0,0,0,1,1,0,0,0,0
1,Perempuan,49cugkYNFakCCkId9Iai,Lagi menunggu ujian sbmptn,21,0,0,1,0,0,0,0,0,0,0
2,Pria,db5wFkLGR4pb7ongAd3e,Kekerasan fisik,22,0,1,0,1,0,0,0,0,0,0
3,Perempuan,gSshnDOP0BbKXuaYTHZI,Kesulitan dalam berkonsentrasi dan mengingat s...,21,0,0,0,0,0,0,1,0,0,0


In [150]:
dict(MAIN_DATAFRAME)

{'Caregiver_Gender': 0         Pria
 1    Perempuan
 2         Pria
 3    Perempuan
 Name: Caregiver_Gender, dtype: object,
 'CAREGIVER_ID': 0     HUDwzgELY1OEzwieoHXK
 1     49cugkYNFakCCkId9Iai
 2     db5wFkLGR4pb7ongAd3e
 3     gSshnDOP0BbKXuaYTHZI
 Name: CAREGIVER_ID, dtype: object,
 'text': 0                                           aku stress
 1                           Lagi menunggu ujian sbmptn
 2                                      Kekerasan fisik
 3    Kesulitan dalam berkonsentrasi dan mengingat s...
 Name: text, dtype: object,
 'Caregiver_Age': 0    21
 1    21
 2    22
 3    21
 Name: Caregiver_Age, dtype: int64,
 'Caregiver-Anxiety': 0    1
 1    0
 2    0
 3    0
 Name: Caregiver-Anxiety, dtype: int32,
 'Caregiver-Depresi': 0    0
 1    0
 2    1
 3    0
 Name: Caregiver-Depresi, dtype: int32,
 'Caregiver-Gangguan-kecemasan': 0    0
 1    1
 2    0
 3    0
 Name: Caregiver-Gangguan-kecemasan, dtype: int32,
 'Caregiver-Gangguan-stress-pascatrauma': 0    0
 1    0
 2   

In [145]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32, text_processed=False):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    if(text_processed):
        processed_text = dataframe.pop('text_processed')
        ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
        ds_processed = tf.data.Dataset.from_tensors(dict(processed_text))
        ds = tf.data.Dataset.zip((ds,ds_processed))
    else:
        ds = tf.data.Dataset.from_tensors(dict(dataframe))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


In [151]:
a =df_to_dataset(MAIN_DATAFRAME)

In [152]:
for feature in a.take(5):
    print(feature)

{'Caregiver_Gender': <tf.Tensor: shape=(1, 4), dtype=string, numpy=array([[b'Pria', b'Perempuan', b'Pria', b'Perempuan']], dtype=object)>, 'CAREGIVER_ID': <tf.Tensor: shape=(1, 4), dtype=string, numpy=
array([[b' HUDwzgELY1OEzwieoHXK', b'49cugkYNFakCCkId9Iai',
        b'db5wFkLGR4pb7ongAd3e', b'gSshnDOP0BbKXuaYTHZI']], dtype=object)>, 'text': <tf.Tensor: shape=(1, 4), dtype=string, numpy=
array([[b'aku stress', b'Lagi menunggu ujian sbmptn', b'Kekerasan fisik',
        b'Kesulitan dalam berkonsentrasi dan mengingat serta ucapan atau perilaku yang tidak teratur']],
      dtype=object)>, 'Caregiver_Age': <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[21, 21, 22, 21]], dtype=int64)>, 'Caregiver-Anxiety': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[1, 0, 0, 0]])>, 'Caregiver-Depresi': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[0, 0, 1, 0]])>, 'Caregiver-Gangguan-kecemasan': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[0, 1, 0, 0]])>, 'Caregiver-Gangguan-str

In [153]:
b = a.map(lambda feature: feature['CAREGIVER_ID'], MAIN_TEXT_MODEL(feature['text']))

ValueError: Exception encountered when calling layer "keras_layer" (type KerasLayer).

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * Tensor("inputs:0", shape=(1, 4), dtype=string)
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (2 total):
    * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (2 total):
    * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
    * True
  Keyword arguments: {}

Call arguments received:
  • args=('tf.Tensor(shape=(1, 4), dtype=string)',)
  • kwargs={'training': 'False'}

In [79]:
tf.data.Dataset.from_tensor_slices([0.234,0.4232])

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float32, name=None)>

In [87]:
data = [0.42, 2.12]
data_np = np.asarray(data, np.float32)

In [88]:
tf.data.Dataset.from_tensor_slices(data_np)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float32, name=None)>

In [89]:
data_np

array([0.42, 2.12], dtype=float32)

In [91]:
np.array([1,2])

array([1, 2])